In [13]:
import os
import shutil
import zipfile
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model
import matplotlib.pyplot as plt

In [14]:
# yes=cats, no=dog
base_dir = 'data/uab_data/'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'val')

# Directory with our training positive pictures
train_pos_dir = os.path.join(train_dir, 'train_primary')

# Directory with our training negative pictures
train_neg_dir = os.path.join(train_dir, 'train_recurrent')

# Directory with our validation positive pictures
validation_pos_dir = os.path.join(validation_dir, 'val_primary')

# Directory with our validation negative pictures
validation_neg_dir = os.path.join(validation_dir, 'val_recurrent')

In [15]:
# Set up matplotlib fig, and size it to fit 4x4 pics
import matplotlib.image as mpimg
nrows = 4
ncols = 4

fig = plt.gcf()
fig.set_size_inches(ncols*4, nrows*4)
pic_index = 100
train_pos_fnames = os.listdir( train_pos_dir )
train_neg_fnames = os.listdir( train_neg_dir )


next_pos_pix = [os.path.join(train_pos_dir, fname)
                for fname in train_pos_fnames[ pic_index-8:pic_index]
               ]

next_neg_pix = [os.path.join(train_neg_dir, fname)
                for fname in train_neg_fnames[ pic_index-8:pic_index]
               ]

for i, img_path in enumerate(next_pos_pix+next_neg_pix):
  # Set up subplot; subplot indices start at 1
  sp = plt.subplot(nrows, ncols, i + 1)
  sp.axis('Off') # Don't show axes (or gridlines)

  img = mpimg.imread(img_path)
  plt.imshow(img)

plt.show()

<Figure size 1152x1152 with 0 Axes>

In [16]:
# Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255.,rotation_range = 40, width_shift_range = 0.2, height_shift_range = 0.2, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator( rescale = 1.0/255. )

In [17]:
# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(train_dir, batch_size = 5, class_mode = 'binary', target_size = (224, 224))

# Flow validation images in batches of 20 using test_datagen generator
validation_generator = test_datagen.flow_from_directory( validation_dir,  batch_size = 5, class_mode = 'binary', target_size = (224, 224))

Found 77 images belonging to 2 classes.
Found 20 images belonging to 2 classes.


In [18]:
from tensorflow.keras.applications.vgg16 import VGG16

base_model = VGG16(input_shape = (224, 224, 3), # Shape of our images
include_top = False, # Leave out the last fully connected layer
weights = 'imagenet')

In [19]:
for layer in base_model.layers:
    layer.trainable = False

In [20]:
base_model.summary()


Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [21]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard

# Flatten the output layer to 1 dimension
x = layers.Flatten()(base_model.output)

# Add a fully connected layer with 512 hidden units and ReLU activation
x = layers.Dense(512, activation='relu')(x)

# Add a dropout rate of 0.5
x = layers.Dropout(0.5)(x)

# Add a final sigmoid layer with 1 node for classification output
x = layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.models.Model(base_model.input, x)

model.compile(optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss = 'binary_crossentropy',metrics = ['acc'])

checkpoint = ModelCheckpoint("methylation_classifier", monitor='acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', save_freq=1)
early = EarlyStopping(monitor='mae', min_delta=0.001, patience=20, verbose=1, mode='auto')
tensorboard = TensorBoard()

In [22]:
folder = 'logs/'
for filename in os.listdir(folder):
    file_path = os.path.join(folder, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))

In [23]:
vgghist = model.fit(train_generator, validation_data = validation_generator, steps_per_epoch = 15, epochs = 300)

model.save("vgg_p_or_r.h5")


Epoch 1/300
15/15 [==============================] - 2s 79ms/step - loss: 1.4108 - acc: 0.5000 - val_loss: 0.6116 - val_acc: 0.6000